<h1 style="font-size:3rem;color:orange;">Getting Started with Basyx Python SDK</h1>
This notebook provides a step-by-step guide for setting up an AAS (Asset Administration Shell) & Submodel, running an API server using the BaSyx Python SDK, and visualizing the data using Dash (web-based visualization)

## Step 1: Create a sample AAS with Submodel and its properties
**Install dependencies (as needed)**

In [2]:

!pip install dash pyqt5 requests


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


**Import Required Libraries**


In [7]:
import plotly.graph_objects as go
import networkx as nx
from basyx.aas import model
from basyx.aas.adapter.json import write_aas_json_file
import json, os

**Get the current working directory to set the storage path dynamically**

In [8]:
current_dir = os.getcwd()

storage_path = os.path.join(current_dir, "server", "app", "storage", "aas_data.json")


**Provide the identifiers and required fields for AAS and submodel**

In [9]:

aas_identifier = "https://example.com/AAS"
submodel_identifier = "https://example.com/Submodel"

asset_info = model.AssetInformation(asset_kind=model.AssetKind.INSTANCE, global_asset_id="Asset1")

aas = model.AssetAdministrationShell(asset_information=asset_info, id_=model.Identifier(aas_identifier))

submodel = model.Submodel(id_=model.Identifier(submodel_identifier))
submodel_reference = model.ModelReference.from_referable(submodel)

aas.submodel.add(submodel_reference)

**Define Properties for the Submodel**

In [10]:


prop1 = model.Property(id_short="Temperature", value_type=model.datatypes.Float, value=25.3, semantic_id=model.ExternalReference(
                (model.Key(
                    type_=model.KeyTypes.GLOBAL_REFERENCE,
                    value='http://acplt.org/Properties/FirstSimpleProperty'
                ),)
            ))
prop2 = model.Property(id_short="Pressure", value_type=model.datatypes.Integer, value=100,  semantic_id=model.ExternalReference(
                (model.Key(
                    type_=model.KeyTypes.GLOBAL_REFERENCE,
                    value='http://acplt.org/Properties/SecondSimpleProperty'
                ),)
            ))
submodel.submodel_element.add(prop1)
submodel.submodel_element.add(prop2)


**Convert the AAS and Submodel data in JSON and Save**

In [11]:

data = model.DictObjectStore()
data.add(aas)
data.add(submodel)
write_aas_json_file(storage_path, data)
print(f"AAS data has been successfully saved to {storage_path} ")

AAS data has been successfully saved to C:\Users\chakrabo\Documents\Project_David\basyx-python-sdk\server\app\storage\aas_data.json 


## Step 2: Host AAS and Submodels via BaSyx AAS Repository and Submodel Repository 
**Import Libraries to Set Up BaSyx AAS and Submodel Server**

In [13]:
import pathlib
import sys, time, threading
import os

from basyx.aas import model, adapter
from basyx.aas.adapter import aasx
from waitress import serve
from basyx.aas.backend.local_file import LocalFileObjectStore
from basyx.aas.adapter.http import WSGIApp


**Set the path and required environment variables**

In [14]:
current_dir = os.getcwd()
storage_path = os.path.abspath(os.path.join(current_dir, "server/app/storage"))
storage_type = os.getenv("STORAGE_TYPE", "LOCAL_FILE_READ_ONLY")
base_path = os.getenv("API_BASE_PATH")

**Host AAS and Submodels by setting up BaSyx Server**

In [15]:
wsgi_optparams = {}

if base_path is not None:
    wsgi_optparams["base_path"] = base_path

if storage_type == "LOCAL_FILE_BACKEND":
    application = WSGIApp(LocalFileObjectStore(storage_path), aasx.DictSupplementaryFileContainer(), **wsgi_optparams)

elif storage_type in "LOCAL_FILE_READ_ONLY":
    object_store: model.DictObjectStore = model.DictObjectStore()
    file_store: aasx.DictSupplementaryFileContainer = aasx.DictSupplementaryFileContainer()

   # with open("storage/aas_data.json") as f:
    with open(os.path.join(storage_path, "aas_data.json")) as f:    
        print(f.read())

    for file in pathlib.Path(storage_path).iterdir():
        if not file.is_file():
            continue
        print(f"Loading {file}")

        if file.suffix.lower() == ".json":
            print(f"DEBUG: Attempting to load {file}")
            with open(file) as f:
                try:
                    adapter.json.read_aas_json_file_into(object_store, f)
                    print(f"DEBUG: Successfully loaded {file}. Object store now contains: {list(object_store)}")
                except Exception as e:
                    print(f"ERROR: Failed to load {file}. Reason: {e}") 
        elif file.suffix.lower() == ".xml":
            with open(file) as f:
                adapter.xml.read_aas_xml_file_into(object_store, file)
        elif file.suffix.lower() == ".aasx":
            with aasx.AASXReader(file) as reader:
                reader.read_into(object_store=object_store, file_store=file_store)

    if not list(object_store):  # If empty, add a sample AAS
        print("No AAS found, adding a default AAS...")
        aas = model.AssetAdministrationShell(
            identification=model.Identifier("ExampleAAS"),
            asset_information=model.AssetInformation(global_asset_id=model.Identifier("ExampleAsset")),
        )
        object_store.add(aas)
    application = WSGIApp(object_store, file_store, **wsgi_optparams)
    

else:
    print(f"STORAGE_TYPE must be either LOCAL_FILE or LOCAL_FILE_READ_ONLY! Current value: {storage_type}",
          file=sys.stderr)
    



{"assetAdministrationShells": [{"modelType": "AssetAdministrationShell", "id": "https://example.com/AAS", "assetInformation": {"assetKind": "Instance", "globalAssetId": "Asset1"}, "submodels": [{"type": "ModelReference", "keys": [{"type": "Submodel", "value": "https://example.com/Submodel"}]}]}], "submodels": [{"modelType": "Submodel", "id": "https://example.com/Submodel", "submodelElements": [{"idShort": "Temperature", "modelType": "Property", "semanticId": {"type": "ExternalReference", "keys": [{"type": "GlobalReference", "value": "http://acplt.org/Properties/FirstSimpleProperty"}]}, "value": "25.3", "valueType": "xs:float"}, {"idShort": "Pressure", "modelType": "Property", "semanticId": {"type": "ExternalReference", "keys": [{"type": "GlobalReference", "value": "http://acplt.org/Properties/SecondSimpleProperty"}]}, "value": "100", "valueType": "xs:integer"}]}]}
Loading C:\Users\chakrabo\Documents\Project_David\basyx-python-sdk\server\app\storage\aas_data.json
DEBUG: Attempting to lo

**Watch for Changes in AAS JSON**

In [16]:
def watch_storage():
    """Monitor storage directory for changes in AAS JSON."""
    json_file_path = os.path.join(storage_path, "aas_data.json")
    
    while not os.path.exists(json_file_path):
        print(f"⚠️ Warning: {json_file_path} does not exist. File watcher will not run.")
        time.sleep(5)
        #return
    last_modified = os.path.getmtime(json_file_path)
    
    while True:
        time.sleep(5)  # Check every 5 seconds
        if os.path.getmtime(json_file_path) > last_modified:
            print("⚠️ Detected changes in AAS JSON. Restarting server...")
            os.execv(sys.executable, ['python'] + sys.argv)
            

**Start the BaSyx Server**

In [17]:
def start_server():
    print("Starting BaSyx AAS Server on http://localhost:8080/api/v3.0")
    serve(application, host="0.0.0.0", port=8080)

server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()

watcher_thread = threading.Thread(target=watch_storage, daemon=True)
watcher_thread.start()

Starting BaSyx AAS Server on http://localhost:8080/api/v3.0


**Once started,  you can visit [http://localhost:8080/api/v3.0/shells](http://localhost:8080/api/v3.0/shells) to verify the AAS data and [http://localhost:8080/api/v3.0/submodels](http://localhost:8080/api/v3.0/submodels) to verify the submodel data.**

## Step 3: Visualization of AAS and submodel data through Dash Web Application
**Import required libraries for Dash Web Application**

In [18]:
import requests
import dash
from dash import dcc, html, Input, Output, dash_table
import pandas as pd

**Fetch & Visualize AAS using Dash**

In [19]:
API_URL = "http://localhost:8080/api/v3.0/shells"

def fetch_aas_data():
    """Fetch AAS data from the Basyx server."""
    for _ in range(5):
        try:
            response = requests.get(API_URL)
            #print(response.status_code)  # Expect 200
            #print(response.json()) 
            data = response.json()
            return data  # Return full AAS data
        except requests.exceptions.ConnectionError:
            print("🔁 AAS server not ready yet. Retrying in 2 seconds...")
            time.sleep(2)
    return {"error": "❌ Failed to fetch AAS data"}

**Fetch & Visualize the submodels using Dash**

In [20]:
API_SUBMODEL_URL = "http://localhost:8080/api/v3.0/submodels"

def fetch_submodel_data():
    """Fetch submodel data from the Basyx server."""
    for _ in range(5):  # Try 5 times
        try:
            response = requests.get(API_SUBMODEL_URL)
            #print(response.status_code)  # Expect 200
            #print(response.json()) 
            data = response.json()
            return data  # Return full submodel data
        except requests.exceptions.ConnectionError:
            print("🔁 AAS server not ready yet. Retrying in 2 seconds...")
            time.sleep(2)
    return {"error": "❌ Failed to fetch Submodel data"}
    

**Create the Dash application instance**

In [21]:
app = dash.Dash(__name__)

**Define the structure of the web application using HTML components**

In [22]:

app.layout = html.Div([
    html.H1("AAS Viewer - Dash Web App"),
    html.Button("Refresh Data", id="refresh-btn", n_clicks=0),
    html.H3("Asset Administration Shells"),
    html.Div(id="aas-output"),
    html.H3("Submodels"),
html.Div(id="submodels-output")   
])


**Callback to Update AAS and Submodels' Output**

In [23]:
@app.callback(
    [Output("aas-output", "children"),
     Output("submodels-output", "children")],
    [Input("refresh-btn", "n_clicks")]
)

def update_view(n_clicks):
    aas_data = fetch_aas_data()
    submodels = fetch_submodel_data()
    
    # Convert JSON data into a readable format
    aas_tree = html.Pre(str(aas_data), style={"whiteSpace": "pre-wrap"})
    submodel_tree = html.Pre(str(submodels), style={"whiteSpace": "pre-wrap"})

    return aas_tree, submodel_tree #return aas_tree, table_data


**Run the Dash App**

In [24]:
if __name__ == "__main__":
    app.run_server(debug=True, port=8050)
